In [1]:
# import libraries
import numpy as np
import matplotlib.pyplot as plt
import time
from keras.callbacks import ModelCheckpoint, EarlyStopping, TensorBoard
from keras.models import load_model
from keras import metrics
from keras.regularizers import l2
import models.nnmodels as nn
import utils.data as data

import gc
gc.enable()
seed = 0
np.random.seed(seed)
%matplotlib inline

Using TensorFlow backend.


In [2]:
print 'Loading DEAP dataset...'
subject_data_folds, valence_folds, arousal_folds = data.load_deap(32, 2)
print 'DEAP dataset is loaded.\n'

print 5*'#' + ' Dataset representation for each subject ' + 5*'#'
print '\nThere are {} subjects in the dataset.\n'.format(len(subject_data_folds))
print '(Trail/Video, Channel, Readings) : ', subject_data_folds[0].shape
print '(Trail/Video, label (valence or arousal)) : ', arousal_folds[0].shape

Loading DEAP dataset...
DEAP dataset is loaded.

##### Dataset representation for each subject #####

There are 32 subjects in the dataset.

(Trail/Video, Channel, Readings) :  (40, 40, 8064)
(Trail/Video, label (valence or arousal)) :  (40, 2)


In [ ]:
 def Cnn1D(num_classes):
 
 
     model = Sequential()
 
     model.add(Conv1D(filters=8, kernel_size=7, strides=2, batch_input_shape=(None, 161280, 1),\
                         kernel_initializer='he_normal',kernel_regularizer=l2(0.1)))
    
 
     model.add(Activation('tanh'))
     model.add(Dropout(0.25))
     #model.add(MaxPooling1D(pool_size=7))
 
     model.add(Conv1D(filters=16, kernel_size=7, strides=2, kernel_initializer='he_normal',\
                     kernel_regularizer=l2(0.1)))
 
     model.add(Activation('tanh'))
     model.add(Dropout(0.25))
     #model.add(MaxPooling1D(pool_size=7))
 

     model.add(AveragePooling1D(pool_size=7))
 
     model.add(Flatten())
 
     model.add(Dense(256, activation='tanh',\
                     kernel_regularizer=l2(0.1)))
 
     model.add(Dropout(0.75))
 
     model.add(Dense(num_classes, activation='softmax'))
 
     adam = Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
 
     model.compile(optimizer=adam, loss='categorical_crossentropy',\
             metrics=['categorical_accuracy'])
     print model.summary()
 
     return model


In [ ]:
from numpy.random import seed
seed(1)
from tensorflow import set_random_seed
set_random_seed(2)
from sklearn.preprocessing import scale
from keras.utils import normalize, print_summary, plot_model
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.optimizers import SGD, RMSprop, Adam
from keras.models import Sequential
from keras.layers import Input, Dense, Dropout, Flatten, Activation, Conv1D, AveragePooling1D, MaxPooling1D



num_subjects = 32
batch_size = 50
epochs = 250
num_exp = 40


accuracies = []

print '##### Two Classes Classification for Valence emotion Using Convolutional Neural Network #####\n'

num_subjects = 32

for s in np.arange(0, num_subjects):
    

    X_val = subject_data_folds[s][:,:,4032:]
    (exp, ch, signals) = X_val.shape
    X_val = X_val.reshape(exp, ch*signals)
    X_val = scale(X_val)
    X_val = np.expand_dims(X_val, axis=2)
    y_val = valence_folds[s]
    
    X_remaining_folds = [x[:,:,4032:] for x in subject_data_folds[:s]] + \
                        [x[:,:,4032:] for x in subject_data_folds[s+1:]]
    y_remaining_folds = valence_folds[:s] + valence_folds[s+1:]
    
    X_train = np.concatenate(X_remaining_folds)
    (exp, ch, signals) = X_train.shape
    X_train = X_train.reshape(exp, ch*signals)
    X_train = scale(X_train)
    X_train = np.expand_dims(X_train, axis=2)
    y_train = np.concatenate(y_remaining_folds)

    model = Cnn1D(num_classes=2)



    key = 's'+str(s+1).zfill(2)+'_valence(3classes)'
    
    

    
    
# checkpoint
    filepath="../saved_models/best_Cnn1D_model_for_{}.h5".format(key)

    checkpoint = ModelCheckpoint(filepath, monitor='val_categorical_accuracy',
                                verbose=1, save_best_only=True, mode='max', period=10)
    
    #tensorboard = TensorBoard(log_dir='../logs', histogram_freq=1, batch_size=50, write_graph=True, \
        #write_grads=True, write_images=True)
    
    callbacks_list = [checkpoint]#, tensorboard]    
    
       
    hist = model.fit(X_train, y_train, 
             batch_size=batch_size,
             epochs=epochs,
             shuffle=True,
             verbose=1,
             validation_data=(X_val, y_val), callbacks=callbacks_list)

    model.load_weights(filepath)
    
    scores = model.evaluate(x=X_val, y=y_val)

    print '#### Result of CNN model for subject {} #####\n'.format(str(s+1).zfill(2))
    print "Loss: {} , Accuracy : {}%".format(scores[0], scores[1]*100)


    accuracies.append(scores[1])
    # summarize history for accuracy
    plt.plot(hist.history['categorical_accuracy'])
    plt.plot(hist.history['val_categorical_accuracy'])
    plt.title('Subject ' +str(s+1).zfill(2)+ ' CNN model accuracy for Valence')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.show()
    
    
    # summarize history for loss
    plt.plot(hist.history['loss'])
    plt.plot(hist.history['val_loss'])
    plt.title('Subject ' +str(s+1).zfill(2)+ ' CNN model loss for Valence')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.show()

##### Two Classes Classification for Valence emotion Using Convolutional Neural Network #####

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_1 (Conv1D)            (None, 80637, 8)          64        
_________________________________________________________________
activation_1 (Activation)    (None, 80637, 8)          0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 80637, 8)          0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 40316, 16)         912       
_________________________________________________________________
activation_2 (Activation)    (None, 40316, 16)         0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 40316, 16)         0         
_______________________________________________

 950/1240 [=====================>........] - ETA: 1s - loss: 0.9891 - categorical_accuracy: 0.6200

In [ ]:
min_acc_subject_idx, max_acc_subject_idx = np.argmin(accuracies), np.argmax(accuracies)

avg_acc =  reduce(lambda x, y: x + y, accuracies) / len(accuracies)

print 5*'#' + ' Accuracies per subject ' + 5*'#'

print 'Subject {} has the minimum accuracy : {}%'.format(min_acc_subject_idx + 1, min(accuracies)*100)
print 'Subject {} has the maximum accuracy : {}%'.format(max_acc_subject_idx + 1, max(accuracies)*100)
print 'Average accuracy : {}%'.format(avg_acc*100)

for subj in enumerate(accuracies):
    print 'Accuracy of Subject {} : {}%'.format(subj[0]+1, subj[1]*100)